# Лабораторна робота № 3

## ІП-14 Бабіч Денис

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import numpy as np
from typing import List

## Створення допоміжних функцій

In [5]:
def show_image(image: np.ndarray, title: str = "") -> None:
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def preprocess_image(image: np.ndarray, kernel_size: int, is_debug: bool = False) -> np.ndarray:
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(grayscale_image, (kernel_size, kernel_size), 0)

    if is_debug:
        show_image(grayscale_image, "Processed grayscale image")
        show_image(blurred_image, "Processed blurred grayscale image")

    return blurred_image

def get_masked_edges(image: np.ndarray, roi_vertices: np.ndarray, is_debug: bool = False) -> np.ndarray:
    ROI_COLOR_POLYGON = (255, 255, 255)
    THRESHOLD_LOW = 75
    THRESHOLD_HIGH = 200

    detected_edges = cv2.Canny(image, THRESHOLD_LOW, THRESHOLD_HIGH)
    mask = np.zeros_like(detected_edges)
    cv2.fillPoly(mask, [roi_vertices], ROI_COLOR_POLYGON)
    masked_image = cv2.bitwise_and(detected_edges, mask)

    if is_debug:
        ROI_OUTLINE_THICKNESS = 2
        ROI_COLOR_OUTLINE = (0, 255, 0)
        debug_image = np.zeros_like(image)

        show_image(detected_edges, "Detected edges")

        cv2.fillPoly(debug_image, [roi_vertices], ROI_COLOR_POLYGON)

        show_image(debug_image, "ROI")

        cv2.polylines(debug_image, [roi_vertices.reshape((-1, 1, 2))], isClosed = True, color = ROI_COLOR_OUTLINE, thickness = ROI_OUTLINE_THICKNESS)

        debug_image = cv2.bitwise_and(image, debug_image)

        show_image(debug_image, "ROI")

    return masked_image

def get_markup_lines(masked_edges: np.ndarray, rho: float, threshold: int, min_length: float, max_gap: float) -> np.ndarray:
    THETA = np.pi / 180
    lines = cv2.HoughLinesP(masked_edges, rho, THETA, threshold, np.array([]), min_length, max_gap)
    return lines

def draw_markup_lines(image: np.ndarray, lines: list, color: List[int] = [0, 255, 0], thickness: int = 3):
    if lines is None or len(lines) == 0:
        return
    
    x_bottom_pos = []
    x_upper_pos = []
    x_bottom_neg = []
    x_upper_neg = []

    y_bottom = image.shape[0]
    y_upper = image.shape[0] - image.shape[0] // 2.1

    for line in lines:
        for x1, y1, x2, y2 in line:
            if (x2 - x1) != 0:
                if ((y2 - y1) / (x2 - x1)) > 0.1 and ((y2 - y1) / (x2 - x1)) < 1.0:
                    slope = ((y2 - y1) / (x2 - x1))
                    b1 = y1 - slope * x1
                    x_bottom_pos.append((y_bottom - b1) / slope)
                    x_upper_pos.append((y_upper - b1) / slope)

                elif ((y2 - y1) / (x2 - x1)) < -0.1 and ((y2 - y1) / (x2 - x1)) > -1.0:
                    slope = ((y2 - y1) / (x2 - x1))
                    b1 = y1 - slope * x1
                    x_bottom_neg.append((y_bottom - b1) / slope)
                    x_upper_neg.append((y_upper - b1) / slope)

    lines_mean = np.array([
      [np.mean(x_bottom_pos), np.mean(y_bottom), np.mean(x_upper_pos), np.mean(y_upper)],
      [np.mean(x_bottom_neg), np.mean(y_bottom), np.mean(x_upper_neg), np.mean(y_upper)]
    ])

    for i in range(len(lines_mean)):
        if not np.isnan(lines_mean[i, 0]) and not np.isnan(lines_mean[i, 1]) and not np.isnan(lines_mean[i, 2]) and not np.isnan(lines_mean[i, 3]):
            cv2.line(image, (int(lines_mean[i, 0]), int(lines_mean[i, 1])), (int(lines_mean[i, 2]), int(lines_mean[i, 3])), color, thickness)

def show_markup(image: np.ndarray, roi_vertices: np.ndarray, rho: float, threshold: int, min_length: float, max_gap: float, color: List[int] = [0, 255, 0], thickness: int = 5, is_debug: bool = False) -> None:
    if is_debug:
        show_image(image, "Unprocessed stock image")
    
    preprocessed_image = preprocess_image(image, 5, is_debug)
    detected_edges = get_masked_edges(preprocessed_image, roi_vertices, is_debug)
    markup_lines = get_markup_lines(detected_edges, rho, threshold, min_length, max_gap)
    draw_markup_lines(image, markup_lines, color, thickness)

---

# Основний етап

## Застосування на прикладі зображення

In [3]:
stock_image = cv2.imread("Data/image.jpg")

RHO = 1
THRESHOLD = 15
MAX_LINES_GAP = 1
MIN_LINES_LENGTH = 1

HEIGHT, WIDTH = stock_image.shape[:2]

HALF_WIDTH = WIDTH // 2
HALF_HEIGHT = HEIGHT // 2
QUARTER_WIDTH = WIDTH // 4
QUARTER_HEIGHT = HEIGHT // 4

roi_vertices = np.array([
    (0, HEIGHT),
    (0, HEIGHT - QUARTER_HEIGHT),
    (HALF_WIDTH, HALF_HEIGHT),
    (WIDTH, HEIGHT - QUARTER_HEIGHT),
    (WIDTH, HEIGHT)
], dtype = np.int32)

show_markup(stock_image, roi_vertices, RHO, THRESHOLD, MIN_LINES_LENGTH, MAX_LINES_GAP, is_debug = True)
show_image(stock_image, "Results")

## Застосування на прикладі відео

In [6]:
ESCAPE_KEY = 27

RHO = 1
THRESHOLD = 7
MAX_LINES_GAP = 5
MIN_LINES_LENGTH = 10

capture = cv2.VideoCapture("Data/video.mp4")
fps = int(capture.get(cv2.CAP_PROP_FPS))

_, frame = capture.read()

HEIGHT, WIDTH = frame.shape[:2]

HALF_WIDTH = WIDTH // 2
HALF_HEIGHT = HEIGHT // 2
QUATER_WIDTH = WIDTH // 4

roi_vertices = np.array([
    (0, HEIGHT),
    (QUATER_WIDTH, HALF_HEIGHT),
    (QUATER_WIDTH * 3, HALF_HEIGHT),
    (WIDTH, HEIGHT)
], dtype = np.int32)

while capture.isOpened():
    ret, frame = capture.read()

    if not ret:
        break

    show_markup(frame, roi_vertices, RHO, THRESHOLD, MIN_LINES_LENGTH, MAX_LINES_GAP, is_debug = False)
    cv2.imshow("Playback", frame)

    if cv2.waitKey(fps) & 0xFF == ESCAPE_KEY:
        break

capture.release()
cv2.destroyAllWindows()